In [12]:
import torch
#import numpy as np
#import torch.nn as nn
#import torch.optim as optim
#import torch.nn.functional as F
#import matplotlib.pyplot as plt
#import torch.utils.data as tud



from func4skipgram import Vocab, wash_chinese
import pickle
import os

# 导入模型定义
from skipgram_model import EmbeddingModel
    

# 读取词典
with open('skipgram_corpus/' + 'meaning' + '.bin', 'rb') as f:
    voca = pickle.load(f)
print('voca读取成功')
# 创建并读取模型
model = EmbeddingModel(len(voca), 50)  # 创建skip gram model
model.load_state_dict(torch.load('models/skipgram_meaning_epoch4_2023-10-15.pth'))
model.eval()


voca读取成功


EmbeddingModel(
  (in_embed): Embedding(9008, 50)
  (out_embed): Embedding(9008, 50)
)

In [13]:
# 评价方法一
C = 4
testsentence = ['机器学习是一门多领域交叉学科，涉及概率论、统计学、逼近论、凸分析、算法复杂度理论等多门学科。专门研究计算机怎样模拟或实现人类的学习行为，以获取新的知识或技能',
                '刘少奇（1898年11月24日-1969年11月12日），生于湖南省宁乡县，伟大的马克思主义者'

]
K = 4
for s in testsentence:
    res = wash_chinese(s)
    pre1 = [[i]*K for i in list(res[:C])] # 概率最大
    likelihood = [0]*C # 可能性
    correct = 0
    for i in range(len(res) - 2*C-1): # 取出中间可以做预测的词
        context = res[i: i+C] + res[i+C+1:i+2*C+1]
        center = res[i+3]
        ids = voca[context]
        ids = torch.IntTensor(ids)  # 只要输入的是列表就没有问题
        pre = model(ids)
        pre = pre.log().sum(dim=0) # 对数和 
        topkpre = torch.topk(pre, K, dim=0, largest=True, sorted=True) # -> (Tensor, LongTensor)
        prop = topkpre[0]
        words = voca.to_tokens(list(topkpre[1]))
        
        #print(words)
        #input()
        if center in words: correct += 1
        pre1.append(words)
        likelihood.append(float(prop.mean(dim=0)))
    print(''.join(res))
    #print(''.join(pre1))
    for i in range(K):
        print(''.join([item[i] for item in pre1]))

    print(f'正确率{correct/(len(res)-2*C):3f}')
    print(f'平均likelihood={sum(likelihood)/len(likelihood):3f}')
    print('\n')




机器学习是一门多领域交叉学科涉及概率论统计学逼近论凸分析算法复杂度理论等多门学科专门研究计算机怎样模拟或实现人类的学习行为以获取新的知识或技能
机器学习的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的学学的学学研的的的的的的的的的的的的的的的的的的的的的的的的
机器学习学学学一有大大学有有学系理理数数论数数析计数数数析析析论论有学学的的学研研学学算研研算计算这有这类类学学学类一学学学作认得得
机器学习大大大有一有学大及交论论学学系大数对系数数析算有数数论理数不文文理理研的究究研计有计计算计有这有有物类生类一为大大大为得可可
机器学习有中中部大学有及与大有理系论理论系相一计算算不不定程数算不文大理院院院院的的技学理算研模机用不是一有物是研作大得得得学为认以
正确率0.111111
平均likelihood=-50.547803


刘少奇（1898年11月24日-1969年11月12日）生于湖南省宁乡县伟大的马克思主义者
刘少奇（1111年11111111111年11111111县县县县县省的县的的的
刘少奇（年年9919月02年年年999919月929年22省今省江江县山省林山义
刘少奇（00年年9年92900月22209年9月9年9年9今的江山南江南郡福林主
刘少奇（9902002902990002月020月22日年于江今今山南东州山为为
正确率0.486486
平均likelihood=-40.890287




In [14]:
# 评价方法二，找出预测概率最高的几个词，判断8个窗口词是否被包含在内
C = 4
testsentence = ['机器学习是一门多领域交叉学科，涉及概率论、统计学、逼近论、凸分析、算法复杂度理论等多门学科。专门研究计算机怎样模拟或实现人类的学习行为，以获取新的知识或技能',
                '刘少奇（1898年11月24日-1969年11月12日），生于湖南省宁乡县，伟大的马克思主义者'

]
K = 10
# torch.topk(input, k, dim=None, largest=True, sorted=True, out=None) -> (Tensor, LongTensor)
#input:tensor数据
#k:排序后的前k个数据
#dim:沿着某个维度
#largest：True是指从大到小，否则，从小到大
#sorted:True返回的结果


for s in testsentence[:1]:
    res = wash_chinese(s)
    preres = {} # 概率最大
    likelihood = [] # 可能性
    correct = 0
    for i in range(len(res) - 2*C-1): # 取出中间可以做预测的词
        context = res[i: i+C] + res[i+C+1:i+2*C+1]
        center = [res[i+C]]
        ids = voca[center] # 取出来的是一个数字
        ids = torch.IntTensor(ids)  # 得到预测
        pre = model(ids)
        
        argm = torch.topk(pre, K, dim=1, largest=True, sorted=True, out=None) # -> (Tensor, LongTensor)
        #print(argm)
        prop = argm[0]
        words = voca.to_tokens(list(argm[1][0]))
        #print(words)
        #print(context)
        for word in context:
            if word in words: 
                correct += 1
        preres[center[0]] = words
        likelihood.append(prop)

    print(f'正确率{correct/((len(res)-2*C)*2*C):3f}')
    #print(f'平均likelihood={sum(likelihood)/len(likelihood):3f}')
    print('\n')
    print(*list(preres.items()), sep='\n')

正确率0.083333


('是', ['的', '这', '鲸', '一', '是', '它', '座', '认', '个', '否'])
('一', ['个', '唯', '般', '座', '些', '的', '样', '种', '第', '次'])
('门', ['屯', '衙', '门', '澳', '鲘', '厦', '阊', '咀', '氹', '锣'])
('多', ['许', '很', '数', '多', '的', '众', '有', '类', '较', '不'])
('领', ['袖', '领', '占', '导', '队', '军', '土', '挥', '域', '羌'])
('域', ['域', '区', '领', '的', '积', '地', '陲', '海', '扩', '邻'])
('交', ['交', '通', '易', '券', '汇', '线', '换', '的', '贸', '路'])
('叉', ['叉', '颥', '肛', '戟', '锏', '梳', '氅', '棘', '尖', '咀'])
('学', ['学', '校', '哲', '院', '习', '医', '科', '教', '硕', '术'])
('科', ['科', '学', '鳄', '鲹', '技', '𬶮', '研', '鹟', '鲨', '蜥'])
('涉', ['嫌', '涉', '贿', '贪', '污', '赂', '问', '及', '案', '件'])
('及', ['汫', '荃', '务', '褂', '涉', '艺', '技', '杯', '澳', '港'])
('概', ['概', '念', '的', '论', '熵', '悖', '数', '这', '函', '值'])
('率', ['率', '频', '降', '低', '量', '锟', '敛', '速', '力', '领'])
('论', ['论', '讨', '悖', '舆', '评', '遑', '辩', '理', '的', '谬'])
('统', ['系', '统', '传', '治', '总', '的', '器', '操', '理', '序'])
('计', ['划', '算', '计', '估', '设', '预', '程', '的', '数', '测'])


In [17]:
# 三，直接输入词，查看这些单个的字模型给出的语境词预测

testwords = ['机器学习', '中间可以做', '丝氨酸蛋白酶'] # 在这里以列表存单个字或词或句的方式放你想测试的token

res = wash_chinese(s)

preres = {} # 概率最大
for token in res: # 取出中间可以做预测的词
    center = token
    ids = voca[center] # 取出来的是一个数字
    ids = torch.IntTensor([ids])  # 得到预测
    pre = model(ids)
    
    argm = torch.topk(pre, K, dim=1, largest=True, sorted=True, out=None) # -> (Tensor, LongTensor)
    #print(argm)
    prop = argm[0]
    words = voca.to_tokens(list(argm[1][0]))
    #print(words)
    #print(context)
    for word in context:
        if word in words: 
            correct += 1
    preres[center[0]] = words


print(*list(preres.items()), sep='\n')

('机', ['机', '械', '飞', '构', '舱', '场', '轰', '车', '牵', '动'])
('器', ['器', '滤', '浏', '伺', '装', '用', '机', '簧', '储', '控'])
('学', ['学', '校', '哲', '院', '习', '医', '科', '教', '硕', '术'])
('习', ['习', '学', '课', '惯', '诂', '练', '训', '读', '校', '术'])
('是', ['的', '这', '鲸', '一', '是', '它', '座', '认', '个', '否'])
('一', ['个', '唯', '般', '座', '些', '的', '样', '种', '第', '次'])
('门', ['屯', '衙', '门', '澳', '鲘', '厦', '阊', '咀', '氹', '锣'])
('多', ['许', '很', '数', '多', '的', '众', '有', '类', '较', '不'])
('领', ['袖', '领', '占', '导', '队', '军', '土', '挥', '域', '羌'])
('域', ['域', '区', '领', '的', '积', '地', '陲', '海', '扩', '邻'])
('交', ['交', '通', '易', '券', '汇', '线', '换', '的', '贸', '路'])
('叉', ['叉', '颥', '肛', '戟', '锏', '梳', '氅', '棘', '尖', '咀'])
('科', ['科', '学', '鳄', '鲹', '技', '𬶮', '研', '鹟', '鲨', '蜥'])
('涉', ['嫌', '涉', '贿', '贪', '污', '赂', '问', '及', '案', '件'])
('及', ['汫', '荃', '务', '褂', '涉', '艺', '技', '杯', '澳', '港'])
('概', ['概', '念', '的', '论', '熵', '悖', '数', '这', '函', '值'])
('率', ['率', '频', '降', '低', '量', '锟', '敛', '速', '力', '领'])
('论', ['论', '讨